# Some applications with Scipy

Ref.: [Scipy doc](https://docs.scipy.org/doc/scipy/reference/)

In [1]:
%matplotlib notebook
from scipy import integrate
import numpy as np
import matplotlib.pyplot as plt

## Planet simulation using gravity

Let's consider two bodies (planets) with respective mass $m_1$ and $m_2$ at positions $P_1$ and $P_2$. Planet 2 will apply a force on planet 1 as follows:

$$
\vec F_{2\rightarrow 1} = G \times \dfrac{m_1 m_2}{r^2} \vec u_{12}
$$

Where:

* $G$ is the gravitational constant,
* and $\vec u_{12}$ is a unit vector coming from $P_1$ and going to $P_2$

So the acceleration of planet $i$ caused by other planets will be:

$$
\vec A(i/0) = \sum_{j\neq i} G \times \dfrac{m_j}{r_{ij}^2} \vec u_{ij}
$$

Let's assume that the position of the planet $i$ is:

$$
\vec{OP_i} = \begin{bmatrix} x_i \\ y_i \\ z_i \end{bmatrix}
$$

The speed is:

$$
\vec V(i/0) = \begin{bmatrix} \dot x_i \\ \dot y_i \\ \dot z_i \end{bmatrix}
$$

The acceleration is:

$$
\vec A(i/0) = \begin{bmatrix} \ddot x_i \\ \ddot y_i \\ \ddot z_i \end{bmatrix}
$$

So the differential equations we need to solve:

$$
\begin{bmatrix} \ddot x_i \\ \ddot y_i \\ \ddot z_i \end{bmatrix}
=
G \sum_{j\neq i}  \dfrac{m_j}{r_{ij}^2} \vec u_{ij}  
$$

With:
* $r_{ij}^2 = (x_i - x_j)^2 + (y_i - y_j)^2 +  (z_i - z_j)^2  $ 
* and:

$$
u_{ij} = \dfrac{\vec {P_iP_j}}{ r_{ij}}
$$


In order to solve the equation numerically, we have to rewrite it as a first order equation in the form:

$$
\dot X = f(X, t)
$$

Here:
$$
X = \begin{bmatrix} x_0 \\ y_0 \\ z_0 \\ \ldots \\ x_{N-1} \\ y_{N-1} \\ z_{N-1} \\ 
    \dot x_0 \\ \dot y_0 \\ \dot z_0 \\ \ldots \\ \dot x_{N-1} \\ \dot y_{N-1} \\ \dot z_{N-1}
    \end{bmatrix}
$$

So 

$$
\dot X = \begin{bmatrix} 
    \dot x_0 \\ \dot y_0 \\ \dot z_0 \\ \ldots \\ \dot x_{N-1} \\ \dot y_{N-1} \\ \dot z_{N-1} \\
    \ddot x_0 \\ \ddot y_0 \\ \ddot z_0 \\ \ldots \\ \ddot x_{N-1} \\ \ddot y_{N-1} \\ \ddot z_{N-1}
    \end{bmatrix}
$$

Let's code it: